# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [2]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [3]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-CRuT4rTkwm75BdF8vnHrTOA78BxVO',
 'object': 'chat.completion',
 'created': 1760767954,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: On Venus, a day (rotation) is longer than a year (orbit around the Sun). A Venus day is about 243 Earth days, while a Venus year is about 225 Earth days. And because Venus rotates retrograde, the Sun rises in the west and sets in the east. Want another fun fact?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 524,
  'total_tokens': 535,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 448,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [4]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: On Venus, a day (rotation) is longer than a year (orbit around the Sun). A Venus day is about 243 Earth days, while a Venus year is about 225 Earth days. And because Venus rotates retrograde, the Sun rises in the west and sets in the east. Want another fun fact?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [5]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: On Venus, a day is longer than a year. A sidereal day on Venus is about 243 Earth days, while its year (orbit around the Sun) is about 224.7 Earth days. Want another fun fact?'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



In [ ]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-pro", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [7]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [ ]:
!ollama pull llama3.2

In [8]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [9]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Here's one:\n\nDid you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! This is because honey has antimicrobial properties that prevent the growth of bacteria, yeast, and mold, making it a near-perfect preservative."

In [6]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏  64 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 151 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 432 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 572 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 719 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.1 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.2 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕         

In [2]:
#response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])
#response.choices[0].message.content

from ollama import chat  # pip install ollama

resp = chat(
    model='deepseek-r1:1.5b',
    messages=[{'role': 'user', 'content': 'Tell me a fun fact'}],
)

print(resp['message']['content'])   # forma dict
# o
print(resp.message.content)         # forma con atributos


Sure! Here's a fun fact for you:

The Earth is not a perfect sphere; it's slightly flattened at the poles. This shape, known as an oblate spheroid, makes the North and South poles appear as if they're heavier than the equator when viewed from above due to centrifugal force caused by its rotation.

How's that cool? 😎
Sure! Here's a fun fact for you:

The Earth is not a perfect sphere; it's slightly flattened at the poles. This shape, known as an oblate spheroid, makes the North and South poles appear as if they're heavier than the equator when viewed from above due to centrifugal force caused by its rotation.

How's that cool? 😎


# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [4]:
# --- Imports y setup ---
import os, re
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

load_dotenv(override=True)

# 1) Cliente OpenAI apuntando a Ollama local
openai = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama",                
)

# 2) Utilidad mínima para traer contenido web 
import requests
from bs4 import BeautifulSoup
def fetch_website_contents(url: str) -> str:
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()
    text = soup.get_text(separator="\n")
    lines = [ln.strip() for ln in text.splitlines()]
    return "\n".join([ln for ln in lines if ln])

# 3) DeepSeek R1 suele devolver un bloque <think>…</think>. Lo limpiamos:
think_re = re.compile(r"<think>.*?</think>\s*", flags=re.DOTALL)
def strip_think(s: str) -> str:
    return think_re.sub("", s).strip()

# 4) Prompts
system_prompt = (
    "You are a helpful assistant that analyzes the contents of a website, "
    "and provides a short, snarky, humorous summary, ignoring navigation text. "
    "Respond in markdown, without wrapping in code fences."
)
user_prefix = (
    "Here are the contents of a website. Provide a short summary. "
    "If it includes news or announcements, summarize those too.\n\n"
)

def messages_for(website: str):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prefix + website},
    ]

# 5) Funciones de resumen + display
def summarize(url: str) -> str:
    website = fetch_website_contents(url)
    resp = openai.chat.completions.create(
        model="deepseek-r1:1.5b",          # ← tu modelo local en Ollama
        messages=messages_for(website),
        temperature=0.3,
        # Puedes ajustar parámetros de Ollama con extra_body → options
        extra_body={"options": {"num_ctx": 4096}},
    )
    text = resp.choices[0].message.content
    return strip_think(text)

def display_summary(url: str):
    display(Markdown(summarize(url)))

# 6) Salida
display_summary("https://edwarddonner.com")



The website is centered around AI-related features and announcements, with several key sections highlighted:

1. **AI Features**: Includes sections like Connect Four, Outsmart, and an interactive "An arena..." game or platform focused on language models.

2. **Cloud Computing**: Mentioned AWS for infrastructure supporting AI projects.

3. **Co-founder Role**: Ed is both a CTO and co-founder of Nebula.io, applying AI to help people discover their potential.

4. **Notable Events**: Specific 2025 dates for AI-related events like the Executive Briefing and Agentic AI Engineering Course.

5. **Contact Information**: The user's email address for direct communication.

6. **Newsletter Section**: For subscribers to stay updated on AI news and developments.

The site is a hub for AI innovation, emphasizing its role in discovery, talent sourcing, and technology advancement.